In [1]:
import pandas as pd
import random
import pickle
from sklearn.preprocessing import MinMaxScaler
from sklearn.cluster import KMeans
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials

### Connecting to Spotify:

In [2]:
secrets_file = open("secrets.txt","r")

string = secrets_file.read()

secrets_dict={}

for line in string.split('\n'):
    if len(line) > 0:
        secrets_dict[line.split(':')[0]]=line.split(':')[1].strip()
        
sp = spotipy.Spotify(auth_manager=SpotifyClientCredentials(client_id=secrets_dict['cid'],
                                                           client_secret=secrets_dict['csecret']))

### Unpickling my Model and Scaler:

In [3]:
kmeans = pickle.load(open('kmeans.pkl','rb'))
scaler = pickle.load(open('minmax.pkl','rb'))

### Loading the Data needed:

In [4]:
top100 = pd.read_csv('songs.csv')
alternative = pd.read_csv('kaggle_and_my_songs.csv')

In [5]:
display(top100.head(3))
display(alternative)

,chart_position,song,artist,genre,release
0,1,Last Night,Morgan Wallen,Country,"February 2, 2023"
1,2,Flowers,Miley Cyrus,Pop,"January 12, 2023"
2,3,Fast Car,Luke Combs,Country,"March 24, 2023"


,song,artist,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature,id,cluster
0,Take Me To Church,Hozier,0.566,0.664,4.0,-5.303,0.0,0.0464,0.6340,0.000000,0.1160,0.4370,128.945,241688.0,4.0,7dS5EaCoMnN7DzlpT6aRn2,25
1,Cooler Than Me - Single Mix,Mike Posner,0.768,0.820,7.0,-4.630,0.0,0.0474,0.1790,0.000000,0.6890,0.6250,129.965,213293.0,4.0,2V4bv1fNWfTcyRJKmej6Sj,16
2,See You Again (feat. Kali Uchis),"Tyler, The Creator",0.558,0.559,6.0,-9.222,1.0,0.0959,0.3710,0.000007,0.1090,0.6200,78.558,180387.0,4.0,7KA4W4McWYRpgf0fWsJZWB,21
3,Pompeii,Bastille,0.679,0.715,9.0,-6.383,1.0,0.0407,0.0755,0.000000,0.2710,0.5710,127.435,214148.0,4.0,3gbBpTdY8lnQwqxNCcf795,17
4,Hips Don't Lie (feat. Wyclef Jean),Shakira,0.778,0.824,10.0,-5.892,0.0,0.0707,0.2840,0.000000,0.4050,0.7580,100.024,218093.0,4.0,3ZFTkvIE7kyPt6Nu3PEa7V,9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
604473,云与海,阿YueYue,0.560,0.518,0.0,-7.471,0.0,0.0292,0.7850,0.000000,0.0648,0.2110,131.896,258267.0,4.0,5rgu12WBIHQtvej2MdHSH0,14
604474,blind,ROLE MODEL,0.765,0.663,0.0,-5.223,1.0,0.0652,0.1410,0.000297,0.0924,0.6860,150.091,153293.0,4.0,0NuWgxEp51CutD2pJoF4OM,27
604475,What They'll Say About Us,FINNEAS,0.535,0.314,7.0,-12.823,0.0,0.0408,0.8950,0.000150,0.0874,0.0663,145.095,187601.0,4.0,27Y1N4Q4U3EfDU5Ubw8ws2,16
604476,A Day At A Time,"Gentle Bones, Clara Benin",0.696,0.615,10.0,-6.212,1.0,0.0345,0.2060,0.000003,0.3050,0.4380,90.029,142003.0,4.0,45XJsGpFTyzbzeWK8VzR8S,13


### Getting a sing to recommend on:

In [6]:
input_song = [input('Enter your favorite songs\n> ').strip()]

Enter your favorite songs
> In the summertime


### Recommending:

In [9]:
import random

song_lst = top100['song'].tolist()
favorites = top100[top100.song.isin(input_song)]
fav_lst = favorites.song.tolist()
track_feats = pd.DataFrame()
# print(fav_lst)
# print(favorites)


if not not fav_lst:
#     print(not not fav_lst)
    print(random.choice(song_lst))
if not fav_lst:
#         track_feats = pd.DataFrame()
    for a in input_song:
        track = sp.search(q=a)
        track_uri = [track['tracks']['items'][0]['uri']]
        print(track['tracks']['items'][0]['artists'][0]['name'])
    #     print(v)
    #     print(track_uri)
    for b in track_uri:
        track_feats = track_feats.append((sp.audio_features(b)),ignore_index=True)
        track_feats = track_feats.select_dtypes(include='number')
        track_feats = pd.DataFrame(scaler.transform(track_feats), columns=track_feats.columns)
        track_cluster = kmeans.predict(track_feats).tolist()
#     for c in track_id:
#         spotify_url = f'https://open.spotify.com/track/'+c
    #     display(track_feats)
    #     display(track_cluster)
    for c in track_cluster:
        rec = pd.DataFrame(alternative[(alternative['cluster'] == c)].sample())
        spotify_url = f'https://open.spotify.com/track/'+rec.iloc[0]['id']
        song = rec.iloc[0]['song']
        artist = rec.iloc[0]['artist']
        print(f'The recommended song is: {song} by {artist}')
        print(f'You can listen to the song here: {spotify_url}')

Mungo Jerry
The recommended song is: Melody For C by Sonny Clark
You can listen to the song here: https://open.spotify.com/track/5333J0hIUJoXfb2S7bRsaW


C:\Users\Customer\AppData\Local\Temp\ipykernel_23004\1803723485.py:23: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  track_feats = track_feats.append((sp.audio_features(b)),ignore_index=True)
